In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from twitterapistuff import *
import json
import io
import sys
from datetime import datetime

try:
    from pymongo import MongoClient
    from pymongo.errors import ConnectionFailure
    from pymongo.errors import WriteError
    has_mongo_client = True
except ImportError:
    has_mongo_client = False


In [ ]:
def mongo_connect():

    try:
        mc = MongoClient('localhost', 27017)
        print("Connected Successfully")
        
        # Get a database handle to a database named "pytest"
        dbh_int = mc["pytest"]
        print("Connected to database")
        
        return dbh_int

    except ConnectionFailure as e:
        sys.stderr.write("Could not connect to MongoDb %s" % e)
        sys.exit(1)


In [ ]:
def mongo_write(data_snippet, dbh):
    
    try:
        dbh.tweetest.insert(data_snippet)
        print("Successfully inserted document.")
        
    except WriteError as we:
        sys.stderr.write("Unable to insert: %s" % we)
        sys.exit(1)


In [ ]:
class listener(StreamListener):

    def on_status(self, status):
        
        tweeDict = {}
        
        tweeDict['userName'] = status.user.screen_name
        tweeDict['userDisplayName'] = status.user.name
        tweeDict['userDescription'] = status.user.description
        tweeDict['userLocation'] = status.user.location
        tweeDict['userFollowers'] = status.user.followers_count

        tweeDict['tweetTimeStamp'] = status.created_at
        tweeDict['tweetTimeStamp_ms'] = int(status.timestamp_ms)
        tweeDict['tweetId'] = status.id
        tweeDict['tweetText'] = status.text
        tweeDict['tweetCoords'] = status.coordinates

        # print("Starting write...")
        mongo_write(tweeDict, dbh)

        # print("Finished write...")
        return(True)

    def on_error(self, status):
        print(status)


In [ ]:
auth = OAuthHandler(cKey, cSecret)
auth.set_access_token(aToken, aSecret)

print("Starting streaming...")

dbh = mongo_connect()

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["brexit"])
